# **multimoda-rs**: Tutorial examples IVUS/OCT to centerline
## Alining from files
The first example will focus on solving the problem of aligning frames within a pullback of either IVUS or OCT images.
We will start with gated IVUS images (systole/diastole) during two different states (e.g. rest/stress).
The .csv files are expected to set up in the following style:
```text
--------------------------------------------------------------------
|      185     |       5.32     |      2.37       |        0.0     |
|      185     |       5.12     |      2.46       |        0.0     |
|      ...     |       ...      |      ...        |        ...     |
```
where the first column is the frame index the point is from, the second to forth are x-, y- and z-coordinates. The naming conventions of the files are diastolic_contours.csv, diastolic_reference_points.csv, ... (see ./data). This is in alignment with the output of the [AIVUS-CAA software](https://https://github.com/AI-in-Cardiovascular-Medicine/AIVUS-CAA).

The first goal is to align the frames within a pullback by translating their centroids to a line and rotating them towards each other minimizing Hausdorff distance of the contours and catheter contours created from the image center. The influence of the catheter (which represents the image center) on the rotation can be adjusted by the number of points passed to catheter. If no catheter should be created just pass n_points=0.

In the same function states are aligned with each other (e.g. systole to diastole) and z-distance are averaged over the two states to have comparable frame positions. If heartrate is very different (e.g. rest to stress) a resampling is performed of the lower heartrate geometry.

Load packages multimodars, and for linking the numpy package.

In [3]:
import os
from pathlib import Path
import multimodars as mm
import numpy as np

# load the provided example data
os.chdir(Path.cwd().parent / "data")

# mode full compares diastole to systole for rest and stress conditions
rest, stress, dia, sys, _ = mm.from_file(
    mode="full", 
    rest_input_path="ivus_rest", 
    stress_input_path="ivus_stress", 
    step_rotation_deg=0.1, 
    range_rotation_deg=90, 
    rest_output_path="output/rest", 
    stress_output_path="output/stress", 
    diastole_output_path="output/diastole", 
    systole_output_path="output/systole", 
    write_obj=True,
    interpolation_steps=28, 
    image_center=(4.5, 4.5),
    radius=0.5,
    n_points=20,
)

# preparing raw data to plot for comparison
rest_dia = np.genfromtxt("ivus_rest/diastolic_contours.csv")
rest_dia_ref_point = np.genfromtxt("ivus_rest/diastolic_reference_points.csv")
rest_sys= np.genfromtxt("ivus_rest/systolic_contours.csv")
rest_sys_ref_point = np.genfromtxt("ivus_rest/systolic_reference_points.csv")

rest_dia_geom_before = mm.numpy_to_geometry(
    contours_arr=rest_dia,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=rest_dia_ref_point,
)
new_contours = []
for contour in rest_dia_geom_before.contours:
    cont = contour.sort_contour_points()
    new_contours.append(cont)
rest_dia_geom_before.contours = np.array(new_contours)

rest_sys_geom_before = mm.numpy_to_geometry(
    contours_arr=rest_sys,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=rest_sys_ref_point,
)
new_contours = []
for contour in rest_sys_geom_before.contours:
    cont = contour.sort_contour_points()
    new_contours.append(cont)
rest_sys_geom_before.contours = np.array(new_contours)

mm.to_obj(rest_dia_geom_before, "output/unprocessed", walls=False, catheter=False, filename_contours="rest_dia.obj")
mm.to_obj(rest_sys_geom_before, "output/unprocessed", walls=False, catheter=False, filename_contours="rest_sys.obj")

Generating geometry for "ivus_rest"
file/path                                          loaded
ivus_rest/diastolic_contours.csv                   true
ivus_rest/diastolic_reference_points.csv           true
ivus_rest/combined_sorted_manual.csv               true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_stress"
file/path                                          loaded
ivus_stress/diastolic_contours.csv                 true
ivus_stress/diastolic_reference_points.csv         true
ivus_stress/combined_sorted_manual.csv             true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_rest"
file/path                                          loaded
ivus_rest/systolic_contours.csv                    true
ivus_rest/systolic_reference_points.csv            true
ivus_rest/combined_sorted_manual.csv               true
geometry pair: systolic geometry generated
Generating geometry for "ivus_stress"
file/path                                  

In [6]:
# Install if needed
%pip install trimesh plotly

# Imports
import trimesh
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def trimesh_to_mesh3d(mesh, color, name):
    """
    Convert a trimesh.Trimesh to a Plotly Mesh3d trace.
    """
    # get vertices and faces
    verts = mesh.vertices
    faces = mesh.faces
    return go.Mesh3d(
        x=verts[:,0], y=verts[:,1], z=verts[:,2],
        i=faces[:,0], j=faces[:,1], k=faces[:,2],
        color=color, opacity=0.6,
        name=name,
        flatshading=True
    )

def plot_pair(before_paths, after_paths, colors, titles):
    """
    before_paths, after_paths: list of two .obj file paths [dia, sys]
    colors: list of two colors (e.g. ['blue','red'])
    titles: [left_title, right_title]
    """
    before_meshes = [trimesh.load(p) for p in before_paths]
    after_meshes  = [trimesh.load(p) for p in after_paths]

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type":"scene"}, {"type":"scene"}]],
        subplot_titles=titles
    )

    for mesh, color, label in zip(before_meshes, colors, ["diastole","systole"]):
        fig.add_trace(
            trimesh_to_mesh3d(mesh, color, f"before_{label}"),
            row=1, col=1
        )
    for mesh, color, label in zip(after_meshes, colors, ["diastole","systole"]):
        fig.add_trace(
            trimesh_to_mesh3d(mesh, color, f"after_{label}"),
            row=1, col=2
        )

    # link camera on both scenes
    camera = dict(
        eye=dict(x=1.5, y=1.5, z=1.0)
    )
    fig.update_layout(
        width=900, height=450,
        # apply same camera to both
        scene_camera=camera,
        scene2_camera=camera,
        # enforce equal scaling on x/y/z for both subplots
        scene=dict(
            aspectmode="data"
        ),
        scene2=dict(
            aspectmode="data"
        ),
        margin=dict(l=0, r=0, t=30, b=0)
    )
    fig.show()

# Paths “before” geometries
before_paths = [
    "output/unprocessed/rest_dia.obj",
    "output/unprocessed/rest_sys.obj",
]

# Paths “after” (processed) meshes
after_paths = [
    "output/rest/mesh_000_rest.obj",    # diastole post
    "output/rest/mesh_029_rest.obj",    # systole post
]

colors = ["royalblue", "firebrick"]

titles = ["Before Processing", "After Processing"]

plot_pair(before_paths, after_paths, colors, titles)

Note: you may need to restart the kernel to use updated packages.


The data is now neatly ordered in pairs (e.g. diastolic and systolic geometry). Every geometry has contours for lumen and walls and a created catheter. The reference point will be used to align the geometry to the centerline. All points corresponding to a contour are also save in a contour struct.

In [7]:
print(f"Example of PyGeometryPair:\n{rest}")
print(f"Example of PyGeometry:\n{rest.dia_geom}")
print(f"Example of PyContour:\n{rest.dia_geom.contours[0]}")
print(f"Example of PyContourPoint:\n{rest.dia_geom.contours[0].points[0]}")


Example of PyGeometryPair:
Diastolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=385, p=0, x=3.57, y=3.67, z=24.54, aortic=false) 
Systolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=319, p=0, x=3.53, y=3.46, z=20.80, aortic=false)
Example of PyGeometry:
Geometry(17 contours, 17 walls), Catheter(17 catheter), Reference Point: Point(f=385, p=0, x=3.57, y=3.67, z=24.54, aortic=false)
Example of PyContour:
Contour(id=0, points=501, centroid=(3.72, 5.25, 3.86))
Example of PyContourPoint:
Point(f=0, p=0, x=3.88, y=7.54, z=3.86, aortic=false)



The four pairs represent all 4 possible comparison in gated images, as for example in coronary artery anomalies (rest pulsatile lumen deformation, stress pulsatile lumen deformation, stress-induced diastolic lumen deformation and stress-induced systolic lumen deformation).

This can also be used for pre-and post-stenting comparison:

In [4]:
_, _, dia, sys, _ = mm.from_file(
    mode="full",
    rest_input_path="ivus_prestent",
    stress_input_path="ivus_poststent",
    rest_output_path="output/stent_rest",
    stress_output_path="output/stent_stress",
    diastole_output_path="output/stent_diastole",
    systole_output_path="output/stent_systole",
    steps_best_rotation=0.3,
    range_rotation_deg=90,
    interpolation_steps=0
    )

# cell – comparison of post-processing meshes
import trimesh
import plotly.graph_objects as go

# reuse the helper from before
def trimesh_to_mesh3d(mesh, color, name):
    verts = mesh.vertices
    faces = mesh.faces
    return go.Mesh3d(
        x=verts[:,0], y=verts[:,1], z=verts[:,2],
        i=faces[:,0], j=faces[:,1], k=faces[:,2],
        color=color, opacity=0.6,
        name=name,
        flatshading=True
    )

# load the two meshes
mesh_dia = trimesh.load("output/stent_diastole/mesh_000_diastolic.obj")
mesh_sys = trimesh.load("output/stent_diastole/mesh_001_diastolic.obj")

# create traces
trace_dia = trimesh_to_mesh3d(mesh_dia, 'royalblue', 'Before (mesh_000)')
trace_sys = trimesh_to_mesh3d(mesh_sys, 'firebrick', 'After (mesh_001)')

# define a canonical camera position
camera = dict(eye=dict(x=1.5, y=1.5, z=1.0))

# build and show figure
fig = go.Figure(data=[trace_dia, trace_sys])
fig.update_layout(
    title="Post-processing: Prestenting vs Poststenting",
    width=600, height=600,
    scene=dict(
        aspectmode="data",    # equal scales on x/y/z
        camera=camera,
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    ),
    margin=dict(l=0, r=0, t=30, b=0)
)
fig.show()


Generating geometry for "ivus_prestent"
file/path                                          loaded
ivus_prestent/diastolic_contours.csv               true
ivus_prestent/diastolic_reference_points.csv       true
ivus_prestent/combined_sorted_manual.csv           true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_poststent"
file/path                                          loaded
ivus_poststent/diastolic_contours.csv              true
ivus_poststent/diastolic_reference_points.csv      true
ivus_poststent/combined_sorted_manual.csv          true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_prestent"
file/path                                          loaded
ivus_prestent/systolic_contours.csv                true
ivus_prestent/systolic_reference_points.csv        true
ivus_prestent/combined_sorted_manual.csv           true
geometry pair: systolic geometry generated
Generating geometry for "ivus_poststent"
file/path                    

However, for these pre- and poststenting comparisons the original from_file approach is computationally more expensive, than using the more flexible .from_array() approach:

In [5]:
# before_arr = np.genfromtxt("ivus_prestent/diastolic_contours.csv", delimiter='\t')
# before_ref = np.genfromtxt("ivus_prestent/diastolic_reference_points.csv", delimiter='\t')
# after_arr = np.genfromtxt("ivus_poststent/diastolic_contours.csv", delimiter='\t')
# after_ref = np.genfromtxt("ivus_poststent/diastolic_reference_points.csv", delimiter='\t')

# before_geom = mm.numpy_to_geometry(before_arr, np.array([]), np.array([]), before_ref)
# after_geom = mm.numpy_to_geometry(after_arr, np.array([]), np.array([]), after_ref)

# pair = mm.from_array(
#     mode="singlepair",
#     geometry_dia=before_geom,
#     geometry_sys=after_geom,
#     output_path="output/stent_comparison",
#     steps_best_rotation=180,
#     range_rotation_deg=60,
# )

Or you can reconstruct a a single 3D geometry from OCT for example.

In [ ]:
oct_raw = np.genfromtxt("oct_single/oct_contours_raw.csv", delimiter=',')
oct_ref = np.genfromtxt("oct_single/oct_ref.csv", delimiter=',')
oct_geom = mm.numpy_to_geometry(
    contours_arr=oct_raw,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=oct_ref,
)

# this has an accuracy of 0.3° since 36 steps for +-6° range
oct_recon, _ = mm.from_array(
    mode="single",
    geometry=oct_geom,
    step_rotation_deg=0.01,
    range_rotation_deg=6,
    image_center=(5.0, 5.0),
    radius=0.5,
    n_points=40,
    sort=False,
    interpolation_steps=0,
    write_obj=False,
)

## Alignment from array
While the alignment from file is one option, the more flexible option is to create Geometries directly from numpy array, and then perform the same operations with these Geometries. It is enough to provide contour coordinates and a reference point for the different states that should be compared.

In [ ]:
dia_cont = np.genfromtxt("fixtures/idealized_geometry/diastolic_contours.csv", delimiter=',')
dia_ref = np.genfromtxt("fixtures/idealized_geometry/diastolic_reference_points.csv", delimiter=',')

sys_cont = np.genfromtxt("fixtures/idealized_geometry/systolic_contours.csv", delimiter=',')
sys_ref = np.genfromtxt("fixtures/idealized_geometry/systolic_reference_points.csv", delimiter=',')

rest_dia = mm.numpy_to_geometry(
    contours_arr=dia_cont,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=dia_ref,
)

rest_sys = mm.numpy_to_geometry(
    contours_arr=sys_cont,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=sys_ref,
)

# Actual function call
rest, (dia_logs, sys_logs) = mm.from_array(
    mode="singlepair", 
    geometry_dia=rest_dia, 
    geometry_sys=rest_sys, 
    ouput_path="output/rest_array",
    steps_best_rotation=0.01,
    range_rotation_deg=60,
    interpolation_steps=0,
)


## Reordering contours
Especially in IVUS images one problem is that breathing leads to bulk movements of frames relative to the catheter. In this case `multimoda-rs` offers the possibility to refine the ordering of the frames, by either providing records with the correct frame ordering or by using the function `refine_ordering`.

In [ ]:
# no example provided yet

## Align with centerline
After aligning the frames within a geometry the alignment with a CCTA centerline can be performed by providing three different points. Here the example of a anomalous coronary artery where a point for the aorta one for the proximal part of the vessel and one for the distal part are provided.

<img src="./figures/Alignment3p.png" alt="Alignment figure" width="500"/>

In [ ]:
rest, (dia_logs, sys_logs) = mm.from_file(
    mode="singlepair", 
    input_path="ivus_rest",
    output_path="output/rest"
)

cl_raw = np.genfromtxt("centerline_raw.csv", delimiter=',')
cl_converted = mm.numpy_to_centerline(cl_raw)
aligned_geometry = mm.align_three_point(
    centerline=cl_converted,
    geometry_pair=rest,
    aortic_ref_pt=(12.2605, -201.3643, 1751.0554),
    upper_ref_pt=(11.7567, -202.1920, 1754.7975),
    lower_ref_pt=(15.6605, -202.1920, 1749.9655),
    write=True,
)




In [ ]:
import trimesh
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def trimesh_to_mesh3d(mesh, color, name):
    """
    Convert a trimesh.Trimesh to a Plotly Mesh3d trace.
    """
    # get vertices and faces
    verts = mesh.vertices
    faces = mesh.faces
    return go.Mesh3d(
        x=verts[:,0], y=verts[:,1], z=verts[:,2],
        i=faces[:,0], j=faces[:,1], k=faces[:,2],
        color=color, opacity=0.6,
        name=name,
        flatshading=True
    )

def plot_pair(before_paths, after_paths, colors, titles):
    """
    before_paths, after_paths: list of two .obj file paths [dia, sys]
    colors: list of two colors (e.g. ['blue','red'])
    titles: [left_title, right_title]
    """
    before_meshes = [trimesh.load(p) for p in before_paths]
    after_meshes  = [trimesh.load(p) for p in after_paths]

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type":"scene"}, {"type":"scene"}]],
        subplot_titles=titles
    )

    for mesh, color, label in zip(before_meshes, colors, ["diastole","systole"]):
        fig.add_trace(
            trimesh_to_mesh3d(mesh, color, f"before_{label}"),
            row=1, col=1
        )
    for mesh, color, label in zip(after_meshes, colors, ["diastole","systole"]):
        fig.add_trace(
            trimesh_to_mesh3d(mesh, color, f"after_{label}"),
            row=1, col=2
        )

    # link camera on both scenes
    camera = dict(
        eye=dict(x=1.5, y=1.5, z=1.0)
    )
    fig.update_layout(
        width=900, height=450,
        # apply same camera to both
        scene_camera=camera,
        scene2_camera=camera,
        # enforce equal scaling on x/y/z for both subplots
        scene=dict(
            aspectmode="data"
        ),
        scene2=dict(
            aspectmode="data"
        ),
        margin=dict(l=0, r=0, t=30, b=0)
    )
    fig.show()

# Paths “before” geometries
before_paths = [
    "output/rest/mesh_000_rest.obj",
    "output/rest/mesh_029_rest.obj",
]

# Paths “after” (processed) meshes
after_paths = [
    "output/aligned/mesh_000_None.obj",    # diastole post
    "output/aligned/mesh_029_None.obj",    # systole post
]

colors = ["royalblue", "firebrick"]

titles = ["Before Alignment", "After Alignment"]

plot_pair(before_paths, after_paths, colors, titles)